In [1]:
#standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json, math, time
from tqdm.notebook import tqdm_notebook
import tmdbsimple as tmdb

In [2]:
with open('/Users/chas/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

In [3]:
tmdb.API_KEY = login['api-key']

In [4]:
#save API call data in data folder created in part 1
import os
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['title_basics.csv.gz',
 '.ipynb_checkpoints',
 'title_akas.csv.gz',
 'title_ratings.csv.gz']

In [7]:
#load dataframe from part 1
basics = pd.read_csv('Data/title_basics.csv.gz')
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0093119,movie,Grizzly II: Revenge,Grizzly II: The Predator,0,2020,NaN,74,"Horror,Music,Thriller"


In [5]:
#Define the years we are currently seeking
YEARS_TO_GET = [2000,2001]

In [6]:
#set up outer loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS',
                          position=0):
    #define the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    #check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    
    #if it does not exist: create it
    if file_exists==False:
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}], f)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
YEAR = 2000
#define the JSON file to store results for year
JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
#check if file exists
file_exists = os.path.isfile(JSON_FILE)

#if it does not exist: create it
if file_exists==False:
    with open(JSON_FILE,'w') as f:
        json.dump([{'imdb_id':0}], f)

In [12]:
#save new year as the current df
df = basics.loc[basics['startYear']==YEAR].copy()
#save movie ids to list
movie_ids = df['tconst'].copy()
movie_ids

9        tt0113026
10       tt0113092
12       tt0115937
13       tt0116391
14       tt0116628
           ...    
76154    tt8327752
76890    tt8553964
77966    tt8907070
78108    tt8954964
79511    tt9412476
Name: tconst, Length: 1403, dtype: object

In [9]:
#load existing data from json into dataframe
previous_df = pd.read_json(JSON_FILE)

In [10]:
#filter out any IDS in JSON_FILE
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

In [14]:
movie_ids.isin(previous_df['imdb_id'])

9        False
10       False
12       False
13       False
14       False
         ...  
76154    False
76890    False
77966    False
78108    False
79511    False
Name: tconst, Length: 1403, dtype: bool

In [15]:
def write_json(new_data, filename):
    with open(filename, 'r+') as file:
        #load existing data
        file_data = json.load(file)
        #choose extend or append
        if (type(new_data)==list) & (type(file_data)==list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        #set files current position at offset
        file.seek(0)
        #convert back to json
        json.dump(file_data, file)

In [17]:
def get_movie_with_rating(movie_id):
    #get the movie and release dates
    movie = tmdb.Movies(movie_id)
    ##contruct output dict
    movie_info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
            movie_info['certification'] = c['certification']
    return movie_info

In [19]:
test = get_movie_with_rating('tt0113026')
test

{'adult': False,
 'backdrop_path': '/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg',
 'belongs_to_collection': None,
 'budget': 10000000,
 'genres': [{'id': 35, 'name': 'Comedy'},
  {'id': 10402, 'name': 'Music'},
  {'id': 10749, 'name': 'Romance'}],
 'homepage': '',
 'id': 62127,
 'imdb_id': 'tt0113026',
 'original_language': 'en',
 'original_title': 'The Fantasticks',
 'overview': 'Two rural teens sing and dance their way through a forbidden romance and a dangerous travelling carnival.',
 'popularity': 2.615,
 'poster_path': '/hfO64mXz3DgUxkBVU7no2UWRP7x.jpg',
 'production_companies': [{'id': 51207,
   'logo_path': None,
   'name': 'Sullivan Street Productions',
   'origin_country': ''},
  {'id': 51208,
   'logo_path': None,
   'name': 'Michael Ritchie Productions',
   'origin_country': ''},
  {'id': 51209,
   'logo_path': None,
   'name': 'Radmin Company, The',
   'origin_country': ''},
  {'id': 60,
   'logo_path': '/oJXpAs4I3W46e4dkaOEzCa4yBko.png',
   'name': 'United Artists',
   'origin_countr

In [20]:
for movie_id in tqdm_notebook(movie_ids_to_get, desc=f'Movies from {YEAR}', 
                             position = 1, leave=True):
    try:
        temp = get_movie_with_rating(movie_id)
        #append/extend results to existing file
        write_json(temp, JSON_FILE)
        time.sleep(0.02)
        
        #if it fails
    except Exception as e:
        continue

Movies from 2000:   0%|          | 0/1517 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [22]:
df = pd.read_json(JSON_FILE)
df

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0035423,0.0,/hfeiSfWYujh6MKhtGTXyK3DD4nN.jpg,None,48000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 14, 'nam...",,11232.0,en,Kate & Leopold,...,76019048.0,118.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,If they lived in the same century they'd be pe...,Kate & Leopold,0.0,6.3,1102.0,PG-13
2,tt0114447,0.0,None,None,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",,151007.0,en,The Silent Force,...,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,They left him for dead... They should have fin...,The Silent Force,0.0,5.0,3.0,NaN
3,tt0118589,0.0,/bLntSfsqUheiUirdV9Ts8znqsTM.jpg,None,22000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",,10696.0,en,Glitter,...,5271666.0,104.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"In music she found her dream, her love, herself.",Glitter,0.0,4.6,115.0,PG-13
4,tt0118652,0.0,/mWxJEFRMvkG4UItYJkRDMgWQ08Y.jpg,None,1000000.0,"[{'id': 9648, 'name': 'Mystery'}, {'id': 27, '...",,17140.0,en,The Attic Expeditions,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,His search for peace of mind... will leave his...,The Attic Expeditions,0.0,5.0,27.0,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,tt0211915,0.0,/9TEUJy5aRP7LaM1LKTfcxVK34JK.jpg,None,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",https://www.miramax.com/movie/amelie/,194.0,fr,Le Fabuleux Destin d'Amélie Poulain,...,173921954.0,122.0,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,One person can change your life forever.,Amélie,0.0,7.9,9665.0,R
148,tt0212048,0.0,None,None,0.0,[],,243522.0,en,Cause Of Death,...,0.0,95.0,[],Released,Guilt Is In The Eye Of The Beholder.,Cause Of Death,0.0,5.0,1.0,NaN
149,tt0212271,0.0,None,None,0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",None,447371.0,en,Lip Service,...,0.0,95.0,[],Released,,Lip Service,0.0,1.0,1.0,R
150,tt0212292,0.0,None,None,0.0,"[{'id': 18, 'name': 'Drama'}]",,88286.0,en,Kwik Stop,...,0.0,110.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Kwik Stop,0.0,6.4,7.0,
